In [1]:
from mesh import *
import pyvista as pv
from itkwidgets import view
import vtk
import vtk.util.numpy_support as np_support

In [2]:
import gmsh


gmsh.initialize()
# gmsh.fltk.initialize()

# Lets create a simple square element:
model = gmsh.model
geo   = model.geo
option= gmsh.option



hight = 1
width = 1
he    = 1.0/5

P1 = geo.addPoint(     -width/2.0,    hight/2.0, 0, he, 1)
P2 = geo.addPoint(     -width/2.0,   -hight/2.0, 0, he, 2)
P3 = geo.addPoint(width-width/2.0,    hight/2.0, 0, he, 3)
P4 = geo.addPoint(width-width/2.0,   -hight/2.0, 0, he, 4)

L1 = geo.addLine(P1,P2) # Left
L2 = geo.addLine(P2,P4) # bottom
L3 = geo.addLine(P4,P3) # right
L4 = geo.addLine(P3,P1) # up

loop1 = geo.addCurveLoop([L1,L2,L3,L4])

surface = geo.addPlaneSurface([loop1])

model.addPhysicalGroup(0, [P2], 200 )
model.addPhysicalGroup(1, [L1], 201 )
model.addPhysicalGroup(1, [L2], 202 )
model.addPhysicalGroup(1, [L3], 203 )
model.addPhysicalGroup(1, [L4], 204 )
model.addPhysicalGroup(2, [surface], 301 )

model.setPhysicalName(0,200,"Pressure")
model.setPhysicalName(1,201,"left")
model.setPhysicalName(1,202,"bottom")
model.setPhysicalName(1,203,"right")
model.setPhysicalName(1,204,"up")
model.setPhysicalName(2,301,"fluid")


geo.synchronize()

# model.mesh.setRecombine(2)
option.setNumber('Mesh.ElementOrder',2)


model.mesh.generate(2)

gmsh.write("test.msh")
# gmsh.fltk.run()

# gmsh.fltk.finalize()

# gmsh.finalize()

In [3]:
Th = mesh.from_gmsh(gmsh)

print(gmsh.model.getDimension())
gmsh.finalize()

2


A builtin ctypes object gave a PEP3118 format string that does not match its itemsize, so a best-guess will be made of the data type. Newer versions of python may behave correctly.


In [4]:
Th.get_number_elements()

92

In [6]:
Th.group_names

{'Pressure': 200,
 'left': 201,
 'bottom': 202,
 'right': 203,
 'up': 204,
 'fluid': 301}

In [7]:
Th.groups

[[0, 200], [1, 201], [1, 202], [1, 203], [1, 204], [2, 301]]

In [8]:

print(Th.get_all_elements_pd().to_string())

   Dim   Type Tag                        Nodes
1    0  point   1                          [1]
2    1  line3   2                    [0, 4, 8]
3    1  line3   3                    [4, 5, 9]
4    1  line3   4                   [5, 6, 10]
5    1  line3   5                   [6, 7, 11]
6    1  line3   6                   [7, 1, 12]
7    1  line3   7                  [1, 13, 17]
8    1  line3   8                 [13, 14, 18]
9    1  line3   9                 [14, 15, 19]
10   1  line3  10                 [15, 16, 20]
11   1  line3  11                  [16, 3, 21]
12   1  line3  12                  [3, 22, 26]
13   1  line3  13                 [22, 23, 27]
14   1  line3  14                 [23, 24, 28]
15   1  line3  15                 [24, 25, 29]
16   1  line3  16                  [25, 2, 30]
17   1  line3  17                  [2, 31, 35]
18   1  line3  18                 [31, 32, 36]
19   1  line3  19                 [32, 33, 37]
20   1  line3  20                 [33, 34, 38]
21   1  line3

In [6]:
pd = []
pd.append(np.sqrt(Th.nodes[:,0]**2+Th.nodes[:,1]**2))
pd.append(Th.nodes)

pd_names = ['Radious','Vals']

In [7]:
grid = Th.to_vtk(pd = pd, pd_names=pd_names)

In [8]:
view(geometries = grid, point_set=Th.nodes)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [9]:
pvgrid = pv.UnstructuredGrid(grid)

In [10]:
pvgrid.save("test_pyvista.vtk",binary=False)

1

In [11]:
ncells = Th.elements[2]['tags'][0].size
cell_type = np.zeros(ncells,dtype = np.int64)
cell_offset = np.zeros(ncells,dtype=np.int64)
npoints_cell = 6

In [12]:
for i in range(ncells):
    cell_type[i] = element_type_map_vtk[Th.elements[2]['types'][0]]
cell_type

array([22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
       22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
       22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22,
       22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22])

In [13]:
sixes = np.full((Th.elements[2]['nodes'][0].shape[0],1),6,dtype = Th.elements[2]['nodes'][0].dtype)

In [14]:
sixes.dtype

dtype('uint64')

In [15]:

cells = np.c_[sixes,Th.elements[2]['nodes'][0]].flatten()

In [16]:
cells

array([  6,  41,  47,  53,  65,  66,  67,   6,  53,  47,  54,  66,  68,
        69,   6,  42,  48,  55,  70,  71,  72,   6,  48,  53,  55,  73,
        74,  71,   6,  54,  45,  56,  75,  76,  77,   6,  43,  45,  54,
        78,  75,  79,   6,  46,  55,  56,  80,  81,  82,   6,  44,  46,
        56,  83,  82,  84,   6,  23,  24,  45,  28,  85,  86,   6,  32,
        33,  46,  37,  87,  88,   6,  14,  15,  47,  19,  89,  90,   6,
         5,   6,  48,  10,  91,  92,   6,   1,  13,  49,  17,  93,  94,
         6,   7,   1,  49,  12,  94,  95,   6,   0,   4,  50,   8,  96,
        97,   6,  34,   0,  50,  39,  97,  98,   6,   3,  22,  51,  26,
        99, 100,   6,  16,   3,  51,  21, 100, 101,   6,   2,  31,  52,
        35, 102, 103,   6,  25,   2,  52,  30, 103, 104,   6,  23,  45,
        63,  86, 105, 106,   6,  32,  46,  64,  88, 107, 108,   6,  14,
        47,  61,  90, 109, 110,   6,   5,  48,  62,  92, 111, 112,   6,
        46,  44,  64,  83, 113, 107,   6,  45,  43,  63,  78, 11

In [17]:
offset = np.arange(0,ncells*(npoints_cell+1),npoints_cell+1,dtype=int)

In [18]:
offset

array([  0,   7,  14,  21,  28,  35,  42,  49,  56,  63,  70,  77,  84,
        91,  98, 105, 112, 119, 126, 133, 140, 147, 154, 161, 168, 175,
       182, 189, 196, 203, 210, 217, 224, 231, 238, 245, 252, 259, 266,
       273, 280, 287, 294, 301, 308, 315, 322, 329, 336, 343, 350, 357,
       364, 371, 378, 385, 392, 399, 406, 413, 420, 427, 434, 441, 448,
       455, 462, 469])

In [19]:
cells

array([  6,  41,  47,  53,  65,  66,  67,   6,  53,  47,  54,  66,  68,
        69,   6,  42,  48,  55,  70,  71,  72,   6,  48,  53,  55,  73,
        74,  71,   6,  54,  45,  56,  75,  76,  77,   6,  43,  45,  54,
        78,  75,  79,   6,  46,  55,  56,  80,  81,  82,   6,  44,  46,
        56,  83,  82,  84,   6,  23,  24,  45,  28,  85,  86,   6,  32,
        33,  46,  37,  87,  88,   6,  14,  15,  47,  19,  89,  90,   6,
         5,   6,  48,  10,  91,  92,   6,   1,  13,  49,  17,  93,  94,
         6,   7,   1,  49,  12,  94,  95,   6,   0,   4,  50,   8,  96,
        97,   6,  34,   0,  50,  39,  97,  98,   6,   3,  22,  51,  26,
        99, 100,   6,  16,   3,  51,  21, 100, 101,   6,   2,  31,  52,
        35, 102, 103,   6,  25,   2,  52,  30, 103, 104,   6,  23,  45,
        63,  86, 105, 106,   6,  32,  46,  64,  88, 107, 108,   6,  14,
        47,  61,  90, 109, 110,   6,   5,  48,  62,  92, 111, 112,   6,
        46,  44,  64,  83, 113, 107,   6,  45,  43,  63,  78, 11

In [20]:
cell_type.dtype

dtype('int64')

In [21]:
points = Th.nodes

In [22]:
offset

array([  0,   7,  14,  21,  28,  35,  42,  49,  56,  63,  70,  77,  84,
        91,  98, 105, 112, 119, 126, 133, 140, 147, 154, 161, 168, 175,
       182, 189, 196, 203, 210, 217, 224, 231, 238, 245, 252, 259, 266,
       273, 280, 287, 294, 301, 308, 315, 322, 329, 336, 343, 350, 357,
       364, 371, 378, 385, 392, 399, 406, 413, 420, 427, 434, 441, 448,
       455, 462, 469])

In [23]:
i=0
nels = 1
cellsi = cells[offset[i]:offset[i+nels]]
cell_typei = cell_type[i:i+nels]
offseti= offset[i:i+nels]

In [24]:
cellsi

array([ 6, 41, 47, 53, 65, 66, 67], dtype=uint64)

In [25]:
cell_typei

array([22])

In [26]:

offseti

array([0])

In [27]:
grid = pv.UnstructuredGrid(offset, cells, cell_type, points,deep=True)

In [28]:
grid["field"] = Th.nodes[:, 0]
grid["value"] = np.c_[Th.nodes,Th.nodes,Th.nodes]

In [29]:
view(geometries=grid)

Viewer(geometries=[{'vtkClass': 'vtkPolyData', 'points': {'vtkClass': 'vtkPoints', 'name': '_points', 'numberO…

In [ ]:
grid.save("lala.vtk",binary=False)

In [ ]:

gridi = pv.UnstructuredGrid(offseti, cellsi, cell_typei, points, deep=True)

In [ ]:
gridi

In [ ]:
A = np.zeros(3)
B = np.full(7,3)

In [ ]:
np.concatenate((A,B),axis=None)

In [ ]:
grid

In [ ]:


vtk_pts = vtk.vtkPoints()
vtk_pts.SetData(np_support.numpy_to_vtk(points,deep=1))

vtk_grid = vtk.vtkUnstructuredGrid()

vtk_grid.SetPoints(vtk_pts)


vtk_grid.Allocate(ncells)

for i in range(1):
    
    vtk_grid.InsertNextCell(cell_type[i], npoints_cell, cells[offset[i]:offset[i+1]])





In [ ]:
vtk_grid.GetNumberOfCells()

In [ ]:
view(geometries = vtk_grid, point_sets=[points])

In [ ]:
grid = pv.UnstructuredGrid(offset, cells, cell_type, points)

In [ ]:
view(geometries = gridi, point_sets=[points])

In [ ]:
view(geometries = grid, point_sets=[points], show_edges=True, cmap="Plasma (matplotlib)")

In [ ]:
points

In [ ]:

grid = pv.UnstructuredGrid(offset, cells, cell_type, points)

# view(geometries=grid, show_edges=True, cmap="Plasma (matplotlib)")
view(geometries = grid,point_sets=[points], show_edges=True, cmap="Plasma (matplotlib)")


In [ ]:


# offset array.  Identifies the start of each cell in the cells array
offset = np.array([0, 9])

# Contains information on the points composing each cell.
# Each cell begins with the number of points in the cell and then the points
# composing the cell
cells = np.array([8, 0, 1, 2, 3, 4, 5, 6, 7, 8, 8, 9, 10, 11, 12, 13, 14, 15])

# cell type array. Contains the cell type of each cell
cell_type = np.array([vtk.VTK_HEXAHEDRON, vtk.VTK_HEXAHEDRON])

cell1 = np.array(
    [
        [0, 0, 0],
        [1, 0, 0],
        [1, 1, 0],
        [0, 1, 0],
        [0, 0, 1],
        [1, 0, 1],
        [1, 1, 1],
        [0, 1, 1],
    ]
)

cell2 = np.array(
    [
        [0, 0, 2],
        [1, 0, 2],
        [1, 1, 2],
        [0, 1, 2],
        [0, 0, 3],
        [1, 0, 3],
        [1, 1, 3],
        [0, 1, 3],
    ]
)

# points of the cell array
points = np.vstack((cell1, cell2))
points = points.astype(np.float64)
# create the unstructured grid directly from the numpy arrays
grid = pv.UnstructuredGrid(offset, cells, cell_type, points)
point_cloud = pv.PolyData(points)
# plot the grid
view(point_sets=[points], show_edges=False, cmap="Plasma (matplotlib)")
# grid.plot(show_edges=True)

In [ ]:
cell_type.dtype

In [ ]:
points.dtype

In [ ]:
grid.points

In [ ]:
import pyvista as pv


cyl = pv.Cylinder()
arrow = pv.Arrow()
sphere = pv.Sphere()
plane = pv.Plane()
line = pv.Line()
box = pv.Box()
cone = pv.Cone()
poly = pv.Polygon()
disc = pv.Disc()


p = pv.Plotter(shape=(3, 3))

# Top row
p.subplot(0, 0)
p.add_mesh(cyl, color="tan", show_edges=True)
p.subplot(0, 1)
p.add_mesh(arrow, color="tan", show_edges=True)
p.subplot(0, 2)
p.add_mesh(sphere, color="tan", show_edges=True)

# Middle row
p.subplot(1, 0)
p.add_mesh(plane, color="tan", show_edges=True)
p.subplot(1, 1)
p.add_mesh(line, color="tan", line_width=3)
p.subplot(1, 2)
p.add_mesh(box, color="tan", show_edges=True)

# Bottom row
p.subplot(2, 0)
p.add_mesh(cone, color="tan", show_edges=True)
p.subplot(2, 1)
p.add_mesh(poly, color="tan", show_edges=True)
p.subplot(2, 2)
p.add_mesh(disc, color="tan", show_edges=True)

# Render all of them
# view(geometries = cone)
p.show()

In [ ]:
pv.__version__